In [1]:
# !pip install transformers
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
# loss 2.8460 - > 0.2065

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer
from tqdm import tqdm
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
data = pd.read_csv('./data/train.csv')

In [4]:
# tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2', eos_token='</s>')
tokenizer = PreTrainedTokenizerFast.from_pretrained('./skt4_29_epoch/', eos_token='</s>')
model = GPT2LMHeadModel.from_pretrained("./skt4_29_epoch/")

/usr/local/lib/python3.8/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
formatted_data = []
for _, row in tqdm(data.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            # 질문과 답변 쌍을 </s> token으로 연결
            input_text = row[q_col] + tokenizer.eos_token + row[a_col] +tokenizer.eos_token
            input_ids = tokenizer.encode(input_text, return_tensors='pt')
            formatted_data.append(input_ids)

644it [00:01, 329.07it/s]


In [7]:
# model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')


In [6]:
CFG = {
    'LR' : 1e-5, # Learning Rate
    'EPOCHS' : 30, # 학습 Epoch
}
model.to(device) # 모델을 GPU단으로 이동

# 모델 학습 설정
optimizer = AdamW(model.parameters(), lr=CFG['LR'])
model.train()

# 모델 학습
for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    progress_bar = tqdm(enumerate(formatted_data), total=len(formatted_data))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")

    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(formatted_data)}")
    model.save_pretrained(f"./skt_5_{epoch}_epoch")
    tokenizer.save_pretrained(f"./skt_5_{epoch}_epoch")

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1 - Avg Loss: 0.0893: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6440/6440 [06:14<00:00, 17.17it/s]


Epoch 1/30, Average Loss: 0.08929090762571634


Epoch 2 - Avg Loss: 0.0704: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6440/6440 [06:14<00:00, 17.19it/s]


Epoch 2/30, Average Loss: 0.07042501249237808


Epoch 3 - Avg Loss: 0.0687: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6440/6440 [04:55<00:00, 21.82it/s]


Epoch 3/30, Average Loss: 0.06873642134115748


Epoch 4 - Avg Loss: 0.0694:  38%|███████████████████████████████████████████████████▎                                                                                  | 2468/6440 [01:45<02:49, 23.49it/s]


KeyboardInterrupt: 